In [11]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import json
import plotly.graph_objects as go

_ : bool = load_dotenv(find_dotenv())

MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")


client:OpenAI = OpenAI()

In [12]:
# import pandas as pd
from typing import Literal
from travel_map import TravelMap



suggestLocations:[dict] = [
                           {
    "name":"Marysville",
    'lat':48.0517637,
    'lon':-122.1770818,
},
    {
        "name":"Perris",
        'lat':33.7825194,
        'lon':-117.22864779999999,
    }
]

mainLocation:dict = suggestLocations[0]

customMap = TravelMap()

def update_travel_map(mainLocation,suggestLocations) -> Literal['updated']:
    print()
    customMap.updateMap(mainLocation, suggestLocations)
    return 'updated'



In [13]:
# creating assistant

assistant = client.beta.assistants.create(
    name="Travel Assistant",
    instructions="""
        -you are a experienced travel assiatant, 
        -your job is to suggest the user best places you can also use map 
    """,
            
    model="gpt-3.5-turbo-1106",
    # model="gpt-4-turbo-preview",

    tools=[
        {"type": "code_interpreter"},
        {
            "type":'function',
            "function":{
                "name":"update_travel_map",
                "description":"""
                    -this function update the locatoins on map, this function uses the mapbox with plotly python
                    
                    -function has two parameters in delimiter
                     `mainLocation` : this  dictionary is  the center of the map (main location from user preference)
                     `suggestLocations` : this is the list of dictionaries(any extra suggest locations)

                    -keys of location object is in delimiter
                     `name`, `lat`, `lon`

                    -always ensure and check pass these two arguments never call functions without arguments
                     function return `updated` which means map is updated.
                    """,
               "parameters": {
                    "type": "object",
                    "properties": {
                    "mainLocation": {"type": "object", "description": "this is the single object"},
                    "suggestLocations": {"type": "object", "description": "this is the list of objects"},
                },
                "required": ["mainLocation",'suggestLocations']
                }

            }
        }
    ],
    #  tool_choice={"type": "function", "function": {"name": "travel_map"}}
)

In [14]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_caUZLvnm3nSpTHbADRs3zvaJ', created_at=1707145193, metadata={}, object='thread')


In [15]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="show me some hostpitals near IBA university karachi city campus in 5km ?"
)

In [16]:
available_functions = {
    "update_travel_map":update_travel_map ,
} 

In [17]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [18]:
import time
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="travel_map","================================================================")
                  
                    if function_to_call.__name__ == "travel_map":
                        
                        response = function_to_call(
                        mainLocation=function_args.get("mainLocation"),
                        suggestLocations=function_args.get("suggestLocations")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        customMap.showMap()
        break  # Exit the loop after processing the completed run
        

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break



queued .....
Run is queued. Waiting...
in_progress .....
Run is queued. Waiting...
requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_3kktd9w5WDmW7CyoFwvGBNg2',
    'function': {'arguments': '{"mainLocation": {"name": "IBA University, Karachi City Campus", "lat": 24.9311, "lon": 67.1125}, "suggestLocations": [{"name": "Hospital 1", "lat": 24.9298, "lon": 67.1056}, {"name": "Hospital 2", "lat": 24.929, "lon": 67.1078}, {"name": "Hospital 3", "lat": 24.9382, "lon": 67.1071}]}',
     'name': 'travel_map'},
    'type': 'function'},
   {'id': 'call_ewfqa69pvYJjVoKg9go8ZouY',
    'function': {'arguments': '{"mainLocation": {"name": "IBA University, Karachi City Campus", "lat": 24.9311, "lon": 67.1125}, "suggestLocations": [{"name": "Hospital 1", "lat": 24.9298, "lon": 67.1056}, {"name": "Hospital 2", "lat": 24.929, "lon": 67.1078}, {"name": "Hospital 3", "lat": 24.9382, "lon": 67.1071}]}',
     'name': 'travel_map'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
<function travel_map at 0x00000201FB09A5C0> True ================================================================
<function travel_map at 0x00000201FB09A5C0> True ================================================================
[{'tool_call_id': 'call_3kktd9w5WDmW7CyoFwvGBNg2', 'output': None}, {'tool_call_id': 'call_ewfqa69pvYJjVoKg9go8ZouY', 'output': None}] >>>>>


BadRequestError: Error code: 400 - {'error': {'message': '2 validation errors for Request\nbody -> tool_outputs -> 0 -> output\n  none is not an allowed value (type=type_error.none.not_allowed)\nbody -> tool_outputs -> 1 -> output\n  none is not an allowed value (type=type_error.none.not_allowed)', 'type': 'invalid_request_error', 'param': None, 'code': None}}